React è un nuovo approccio che combina reasoning (chain-of-thoughts prompting) e acting da parte degli LLM.

Con ReAct gli LLM generano tracce di ragionamento e azioni specifiche.<br>
Consente di superare alcune limitazioni degli LLM come allucinazioni e propagazione di errori

I Langchain agent consentono di combinare :
- chain
- tools

Un REAct Agent è in grado di decidere, sulla base dell'input fornito all'LLM , qual'è il tool da utilizzare.

E' necessario installare langchain hub che consente di accedere a pretrained prompts, agents e chains

In [1]:
pip install -q langchainhub

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools import PythonREPLTool
from langchain_openai import ChatOpenAI


In [5]:
llm = ChatOpenAI(
        api_key='545454545',
        base_url='http://localhost:8000/v1',
        temperature=0
    )

In [9]:
# Preparo il template che struttura come sono poste le domande

template = """ 
Answer the following questions the best you can-
Questions:{q}
"""

prompt_template = PromptTemplate.from_template(template)
prompt = hub.pull("hwchase17/react")
type(prompt)

langchain_core.prompts.prompt.PromptTemplate

Verifico quali sono le variabili di input del prompt

In [8]:
prompt.input_variables

['agent_scratchpad', 'input', 'tool_names', 'tools']

Visualizzo il testo del template

In [12]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


Utilizzeremo 3 Tools:
- Python Repl Tool : per eseguire codice Python
- Wikipedia Tool : per cercare su wikipedia
- DuckduckGo Toll : per cerca sul web

e l'agent sceglierà di utilizzare il tool corretto sulla base della richiesta dell'utente

In [42]:
# Python Repl Toll
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name="Python Repl",
    func=python_repl.run,
    description="Usefull when you need to use Python to answer a question. You should input Python code."
)

In [52]:
# Wikipedia Tool
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name = "wikipedia",
    func=wikipedia.run,
    doc_content_chars_max=2000,
    description="Usefull when you need to look up a topic, country or person on wikipedia."
)

In [44]:
# DukDukGo Tool
search = DuckDuckGoSearchRun()
dukdukgo_tool = Tool(
    name="DuckDuckGo Search",
    func=search.run,
    max_results=1,
    description="Useful for when you need to perform an internet search to find information that another tool can't provide"

)

In [45]:
# Creo una lista con i tools creati

tools = [python_repl_tool, wikipedia_tool, dukdukgo_tool]

# Creo il React Agent
agent = create_react_agent(llm, tools, prompt)

# Creo AgentExecutor per eseguire l'agent
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,  # Controlla come l'executor gestiesce i parsing error (ad es, debugging)
                                # in caso di False viene restituito errore in caso di parsing error
    max_iterations=3   # Massimo numero di iterazioni per evitare che venga eseguito indefinitamente
)

## TEST ESECUZIONE

Dopo aver definito la domanda, utilizzo l'executor per invocare l'agent

In [63]:
question = "write a python program to calculate the first 20 number in fibonaci sequence"

output=agent_executor.invoke({
    'input' : prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
 Action: Python Repl
Action Input: "def fibonacci(n):\n\treturn [0, 1] + [sum(x, y) for x in range(2, n) for y in range(x, n)]"SyntaxError('unexpected character after line continuation character', ('<string>', 1, 19, 'def fibonacci(n):\\n\\treturn [0, 1] + [sum(x, y) for x in range(2, n) for y in range(x, n)]\n', 1, 0)) Action: Python Repl
Action Input: "def fibonacci(n):\n\treturn [0, 1] + [sum(x, y) for x in range(2, n) for y in range(x, n)]"SyntaxError('unexpected character after line continuation character', ('<string>', 1, 19, 'def fibonacci(n):\\n\\treturn [0, 1] + [sum(x, y) for x in range(2, n) for y in range(x, n)]\n', 1, 0)) Action: Python Repl
Action Input: "def fibonacci(n):\n\treturn [0, 1] + [sum(x, y) for x in range(2, n) for y in range(x, n)]"SyntaxError('unexpected character after line continuation character', ('<string>', 1, 19, 'def fibonacci(n):\\n\\treturn [0, 1] + [sum(x, y) for x in range(2, n) for y in range(x, n)]\n', 1, 

In [33]:
print(output)

{'input': ' \nAnswer the following questions the best you can-\nQuestions:write a python program to show the content of the execution directory\n', 'output': 'Agent stopped due to iteration limit or time limit.'}


In [35]:
output['output']

'Agent stopped due to iteration limit or time limit.'

In [46]:
question = 'Tell me the name of the current prime minister of UK.'

output=agent_executor.invoke({
    'input' : prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
 Action: wikipedia
Action Input: "current prime minister of UK"Page: Prime Minister of the United Kingdom
Summary: The prime minister of the United Kingdom is the head of government of the United Kingdom. The prime minister advises the sovereign on the exercise of much of the royal prerogative, chairs the Cabinet, and selects its ministers. As modern prime ministers hold office by virtue of their ability to command the confidence of the House of Commons, they sit as members of Parliament.
The office of prime minister is not established by any statute or constitutional document, but exists only by long-established convention, whereby the monarch appoints as prime minister the person most likely to command the confidence of the House of Commons. In practice, this is the leader of the political party that holds the largest number of seats in the Commons. The prime minister is ex officio also First Lord of the Treasury (prior to 1905 also the officia

In [48]:
output

{'input': ' \nAnswer the following questions the best you can-\nQuestions:Tell me the name of the current prime minister of UK.\n',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [53]:
question = 'Tell me about Napoleone Bonaparte early life.'

output=agent_executor.invoke({
    'input' : prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
 Action: wikipedia
Action Input: "Napoleone Bonaparte"Page: Napoleon
Summary: Napoleon Bonaparte (born Napoleone di Buonaparte; 15 August 1769 – 5 May 1821), later known by his regnal name Napoleon I, was a French military officer and statesman who rose to prominence during the French Revolution and led a series of successful campaigns across Europe during the French Revolutionary and Napoleonic Wars from 1796 to 1815. He was the leader of the French Republic as First Consul from 1799 to 1804, then of the French Empire as Emperor of the French from 1804 to 1814, and briefly again in 1815.
Born on the island of Corsica to a family of Italian origin, Napoleon moved to mainland France in 1779 and was commissioned as an officer in the French Royal Army in 1785. He supported the French Revolution in 1789, and promoted its cause in Corsica. He rose rapidly through the ranks after winning the siege of Toulon in 1793 and defeating royalist insurgents in 

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 2048 tokens. However, you requested 2768 tokens (2768 in the messages, None in the completion). Please reduce the length of the messages or completion.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [ ]:
output

In [57]:
print(prompt_template.format(q=question))

 
Answer the following questions the best you can-
Questions:Tell me about Napoleone Bonaparte early life.



In [59]:
print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'
